Importations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Installations

In [ ]:
!pip install gdown

## Validations sur le réseau de surface (RDS) : bus, tramway ##

In [ ]:
df_rds = pd.read_csv("data/calidations/validations_rds.csv",sep = ";")
df_rds.head(10)

On sélectionne le tramway T2

In [ ]:
df_t2 = df_rds[df_rds['ID_GROUPOFLIGNE']== "A01192"] #équivalent du WHERE, on sélectionne le T2 avec son ID unique
val_jour_t2 = df_t2.groupby("JOUR")["NB_VALD"].sum() #nb de validations sur chaque jour
val_jour_t2

#ça l'a converti en Series

Lignes du RDS les plus fréquentées

In [ ]:
vald_rds = (
    df_rds.groupby(["JOUR",'ID_GROUPOFLIGNE', 'LIBELLE_LIGNE'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)
vald_rds['NB_VALD_MOY_JOUR'] = (vald_rds['NB_VALD']/91).round(0) #pour arrondir : round(0)
vald_rds.head(10)

#vald_rds[vald_rds["LIBELLE_LIGNE"].astype(str).str.contains("25")]
#vald_rds[vald_rds["LIBELLE_LIGNE"] == "96"]

In [ ]:
vald_rds_1404 = vald_rds.loc[vald_rds["JOUR"] == "2025-04-14"]
vald_rds_1404.head(10)

Profil horaire

In [ ]:
df_ph_rds = pd.read_csv("data/validations/profil_horaire_rds.csv",sep = ";")
df_ph_rds.head(5)

## Offre de transport (GTFS) ##

In [12]:
#calcul de l'offre. Dans le GTFS il y a réseau ferré et réseau de surface.
agency = pd.read_csv("data/gtfs/agency.txt") #associe agency_id avec réseau de transport. Go trouver Paris-Saclay
routes = pd.read_csv("data/gtfs/routes.txt") #associe route_id avec une ligne de transport
trips = pd.read_csv("data/gtfs/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id
calendar = pd.read_csv("data/gtfs/calendar.txt") #quel jour il y a tel service, identifié par service_id
calendar_dates = pd.read_csv("data/gtfs/calendar_dates.txt") #services à ajouter/retirer car exceptions
stops = pd.read_csv("data/gtfs/stops.txt") #associe stop_id avec un arrêt physique

""" 
!!! Lien entre GTFS et données de validation : route_id = IDFM:ID_ligne
 
Les arrêts aussi peuvent être identifiés facilement comme ça si tu as envie
"""

"""
merge : à gauche c'est ta base, ce que tu gardes. A droite, c'est ce que tu ajoutes
"""


/tmp/ipykernel_2741/2933175917.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("data/gtfs/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id


"\nmerge : à gauche c'est ta base, ce que tu gardes. A droite, c'est ce que tu ajoutes\n"

In [ ]:
#Cellule pour lire stop_times.txt stocké sur Google Drive car trop volumineux pour être stocké sur le dépôt GitHub
import os, gdown

#lien : https://drive.google.com/file/d/1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS/view?usp=sharing
file_id = "1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS"
url = f"https://drive.google.com/uc?id={file_id}"
os.makedirs("data", exist_ok=True)
out_path = "data/GTFS/stop_times.txt"
gdown.download(url, out_path, quiet=False)  # gère aussi les gros fichiers >100 Mo

stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id
stop_times.head(10)

Jointures GTFS

In [ ]:
agency_routes = routes.merge(agency,left_on = "agency_id",right_on = "agency_id",how="left")

stop_time_stop = stops.merge(stop_times,left_on="stop_id",right_on="stop_id",how="left")

saclay_gtfs = agency_routes[agency_routes["agency_id"] == "IDFM:1060"] #agency_id : IDFM:1060 pour réseau Paris-Saclay
saclay_gtfs = saclay_gtfs[["agency_id","route_id","route_long_name"]]

trips_saclay = saclay_gtfs.merge(trips,left_on="route_id",right_on="route_id",how="left")
trips_saclay = trips_saclay[["route_id","trip_id","service_id","direction_id","route_long_name"]]

calendar_saclay = trips_saclay.merge(calendar,left_on="service_id",right_on="service_id",how="left")

horaires_saclay = calendar_saclay.merge(stop_time_stop,left_on="trip_id",right_on="trip_id",how="left")
horaires_saclay = horaires_saclay[["route_long_name","trip_id","stop_name","arrival_time","departure_time","monday"]]
horaires_4606 = horaires_saclay[horaires_saclay["route_long_name"] == "4606 (ex 91.06)"]


In [ ]:
horaires_4606_lundi = horaires_4606[horaires_4606["monday"] == 1]
horaires_4606_lundi["trip_id"].nunique()

Offre "les lundis"

In [ ]:
#compter combien de trips : un trip = une course, donc Terminus1 -> Terminus2
#Ici on a compté le nombre de trips "le lundi", et pas sur un lundi représentatif. Donc on surestime probablement l'offre.
horaires_saclay_lundi = horaires_saclay[horaires_saclay["monday"]==1]

trip_par_ligne_lundi = (horaires_saclay_lundi.groupby(
    ["route_long_name"],as_index = False)["trip_id"].nunique()
)
trip_par_ligne_lundi.head(10)

Offre sur un jour précis

In [ ]:
services_jour = (calendar.groupby(
    ["start_date"],as_index=False)["service_id"].nunique()
) #nombre de services qui commencent tel jour

#On prend le lundi 29 septembre, il a l'air bien au milieu de tout

date = pd.Timestamp("2025-09-29")
dint = int(date.strftime("%Y%m%d")) #conversion au même format que les dates des données GTFS

#Services actifs le 29/09
actifs = calendar[
    (calendar["start_date"] <= dint) &
    (calendar["end_date"]   >= dint) &
    (calendar["monday"] == 1)
]["service_id"]

#Exceptions du 29/09
exceptions = calendar_dates[calendar_dates["date"] == dint]
actifs = pd.Index(actifs).unique()

enlever = pd.Index(exceptions.loc[exceptions["exception_type"] == 2, "service_id"]).unique()
ajouter = pd.Index(exceptions.loc[exceptions["exception_type"] == 1, "service_id"]).unique()
#loc : sert à sélectionner certaines lignes et certaines colonnes, avec potentiellement des booléens

actifs = actifs.difference(enlever)
actifs = actifs.union(ajouter)
d_actifs = {}
for service_id in actifs :
    d_actifs[service_id] = 1

In [ ]:
trips_2909 = trips.loc[trips["service_id"].isin(actifs)]
#isin : vérifie pour chaque élément du DataFrame s'il est dans actifs

offre_2909 = trips_2909.merge(routes,on="route_id",how="left")
offre_2909 = offre_2909[["route_id","route_long_name","trip_id","direction_id"]]
offre_2909.head(10)

In [ ]:
#Heure (ou minute) de départ du trip au premier arrêt
st = stop_times.loc[stop_times["trip_id"].isin(offre_2909["trip_id"])].copy()
st = st.sort_values(["trip_id", "stop_sequence"])
first_dep = st.groupby("trip_id", as_index=False).first()[["trip_id", "departure_time"]]

#Conversion de HH:MM:SS en minutes depuis 00:00 (gère par ex "25:10:00" si nécessaire)
def to_min(t):
    #marche même si t dépasse 24:00:00
    h, m, s = map(int, t.split(":"))
    return h*60 + m + s/60

first_dep["dep_min"] = first_dep["departure_time"].map(to_min)

dep = first_dep.merge(offre_2909[["trip_id","route_id","direction_id"]], on="trip_id", how="left")

#Pointe du matin 7h–10h
peak = dep[(dep["dep_min"] >= 7*60) & (dep["dep_min"] < 10*60)]

freq = (peak.groupby(["route_id","direction_id"], as_index=False)
             .size()
             .rename(columns={"size": "voyages_3h"}))
freq["voyages_h"] = freq["voyages_3h"] / 3.0
freq["headway_min"] = 60.0 / freq["voyages_h"]

freq = freq.merge(
    routes[["route_id","route_type","route_long_name"]],
    on="route_id", how="left"
)

In [ ]:
freq_triée = freq.sort_values(by="voyages_h",ascending=False)
"""
with pd.option_context(
    "display.max_rows", 1000,
    "display.max_columns", None,
    "display.width", None,
    "display.max_colwidth", None
):
    display(freq_triée.head(200))

"""

freq_triée_rds = freq_triée[(freq_triée["route_type"] == 0) | (freq_triée["route_type"] == 3)]
freq_triée_rds.head(10)

In [ ]:
freq_routes = freq.merge(routes,on="route_id",how="left")
freq_saclay = freq_routes[freq_routes["agency_id"] == "IDFM:1060"]
freq_4606 = freq_saclay[freq_saclay["route_long_name_x"] == "4606 (ex 91.06)"]

freq_4606[["route_long_name_x","direction_id","voyages_h","headway_min"]] 
#on trouve un bus tous les 4 min en pointe sur le 4606, c'est cohérent !!

In [ ]:
profil_h_rds = df_ph_rds[["LIBELLE_LIGNE","ID_GROUPOFLIGNE","TRNC_HORR_60","Pourcentage_validations","CAT_JOUR"]]
profil_h_rds = profil_h_rds[profil_h_rds["CAT_JOUR"] == "JOHV"] #on s'intéresse aux jours ouvrés hors vacances (JOHV)
profil_h_rds

#il n'y a pas les jours précis sur le profil horaire, seulement des catégories de jours

In [ ]:
profil_7_10 = (profil_h_rds.loc[profil_h_rds["TRNC_HORR_60"].isin(["7H-8H","8H-9H","9H-10H"])]
               .groupby("ID_GROUPOFLIGNE", as_index=False)["Pourcentage_validations"].sum()
               .rename(columns={"Pourcentage_validations": "part_7_10"}))
profil_7_10["part_7_10"] = profil_7_10["part_7_10"]*0.01

vald_prof_rds = vald_rds.merge(profil_7_10,on = "ID_GROUPOFLIGNE",how="left")
vald_prof_rds["NB_VALD_HP"] = vald_prof_rds["NB_VALD_MOY_JOUR"]*vald_prof_rds["part_7_10"]

vald_prof_rds = vald_prof_rds[["ID_GROUPOFLIGNE","LIBELLE_LIGNE","NB_VALD_MOY_JOUR","part_7_10","NB_VALD_HP"]]
vald_prof_rds = vald_prof_rds.rename(columns={'ID_GROUPOFLIGNE' : 'ID_GroupOfLines'})
vald_prof_rds.head(20)

In [ ]:
#Pour les validations on prend le lundi 14/04 comme jour de référence.
vald_prof_rds_1404 = vald_rds_1404.merge(profil_7_10,on = "ID_GROUPOFLIGNE",how="left")
vald_prof_rds_1404 = vald_prof_rds_1404[["ID_GROUPOFLIGNE","LIBELLE_LIGNE","part_7_10","NB_VALD"]]
vald_prof_rds_1404 = vald_prof_rds_1404.rename(columns={'ID_GROUPOFLIGNE' : 'ID_GroupOfLines'})

vald_prof_rds_1404["NB_VALD_HP"] = (vald_prof_rds_1404["NB_VALD"]*vald_prof_rds_1404["part_7_10"]).round(0)
vald_prof_rds_1404.head(10)

Fichier ref_lignes pour lier GTFS et données de validations

In [ ]:
ref_lignes = pd.read_csv("données/ref_lignes.csv",sep=";")
ref_lignes["route_id"] = "IDFM:" + ref_lignes["ID_Line"]
ref_lignes = ref_lignes[["route_id","ID_GroupOfLines"]]

freq_triée_rds = freq_triée_rds.merge(ref_lignes,on="route_id",how="left")
freq_triée_rds

In [ ]:
#On merge GTFS (29/09) et validations (14/04)

saturation_rds = freq_triée_rds.merge(vald_prof_rds_1404,left_on = "ID_GroupOfLines",right_on = "ID_GroupOfLines",how="left")
saturation_rds["NB_VALD_PAR_TRIP"] = (saturation_rds["NB_VALD_HP"]/saturation_rds["voyages_3h"]).round(0)
saturation_rds = saturation_rds[["LIBELLE_LIGNE","route_long_name","direction_id","part_7_10","NB_VALD_HP","voyages_3h","NB_VALD_PAR_TRIP"]]
saturation_rds = saturation_rds.sort_values(by="NB_VALD_PAR_TRIP",ascending=False)

with pd.option_context(
    "display.max_rows", 1000,
    "display.max_columns", None,
    "display.width", None,
    "display.max_colwidth", None
):
    display(saturation_rds.head(100))

"""
Dans ce qu'on obtient en dessous, les nombres semblent cohérents pour les lignes très fréquentées (tramways, TVM...)
mais il semble aussi y avoir de gros problèmes, 1900 c'est beaucoup trop élevé (surtout pour une ligne osef).
Cela pourrait être dû à des problèmes de jointures qui sous-estiment le nombre de trips
"""

## Réseau ferré (RF) : métro, RER, transilien ##

In [6]:
#Données du réseau ferré
import pandas as pd
df_rf = pd.read_csv("data/validations/validations_rf.csv",sep=";")

Stations les plus fréquentées du réseau ferré

In [42]:
vald_rf = (
    df_rf.groupby(['JOUR','ID_ZDC', 'LIBELLE_ARRET'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)

vald_rf_1404 = vald_rf.loc[vald_rf["JOUR"] == "2025-04-14"]

vald_rf['NB_VALD_MOY_JOUR'] = (vald_rf['NB_VALD']/91).round(0) #pour arrondir : round(0)

vald_rf.head(10)

,JOUR,ID_ZDC,LIBELLE_ARRET,NB_VALD,NB_VALD_MOY_JOUR
62410,2025-06-21,71264.0,CHATELET,299725.0,3294.0
7379,2025-04-10,71370.0,SAINT-LAZARE,292283.0,3212.0
59377,2025-06-17,71370.0,SAINT-LAZARE,290122.0,3188.0
64758,2025-06-24,71370.0,SAINT-LAZARE,290064.0,3188.0
54010,2025-06-10,71370.0,SAINT-LAZARE,289518.0,3182.0
60915,2025-06-19,71370.0,SAINT-LAZARE,288136.0,3166.0
5844,2025-04-08,71370.0,SAINT-LAZARE,285898.0,3142.0
66298,2025-06-26,71370.0,SAINT-LAZARE,285609.0,3139.0
22661,2025-04-30,71370.0,SAINT-LAZARE,283379.0,3114.0
21892,2025-04-29,71370.0,SAINT-LAZARE,280907.0,3087.0


Offre sur le réseau ferré

In [13]:
# === 1) Choisir un jour RÉEL et déterminer les service_id actifs ===
date = pd.Timestamp("2025-09-29")  #lundi 29/09
dint = int(date.strftime("%Y%m%d"))

active = calendar[
    (calendar["start_date"] <= dint) &
    (calendar["end_date"]   >= dint) &
    (calendar["monday"] == 1)
]["service_id"]

# Exceptions (ajouts/suppressions) ce jour-là
exc = calendar_dates[calendar_dates["date"] == dint]
to_remove = set(exc.loc[exc["exception_type"] == 2, "service_id"])
to_add    = set(exc.loc[exc["exception_type"] == 1, "service_id"])

active = set(active) - to_remove
active |= to_add

# Harmoniser types
trips["service_id"] = trips["service_id"].astype(str)
active_ids = pd.Index(sorted(map(str, active)))

# === 2) Filtrer l’offre ferroviaire (route_type) et les trips actifs ===
# Ferroviaire classique : route_type == 2, Métro : route_type == 1
rail_routes = routes[routes["route_type"].isin([2,1])] 
trips_day = trips[trips["service_id"].isin(active_ids) &
                  trips["route_id"].isin(rail_routes["route_id"])]

# === 3) Lier aux stop_times et calculer l’heure de passage ===
# On compte un passage par (trip_id, stop_id) : distinct trip_id à un stop
# Utilise arrival_time OU departure_time (une seule, pour ne pas double-compter).
stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id
st = stop_times[stop_times["trip_id"].isin(trips_day["trip_id"])].copy()

# Conversion robuste HH:MM:SS -> minutes, en gérant >24:00:00
def to_min(t):
    if not isinstance(t, str) or t.count(":") != 2:
        return None
    h, m, s = map(int, t.split(":"))
    return h*60 + m + s/60

# Choix de la colonne temps (arrival_time conseillé pour "passage")
st["t_min"] = st["arrival_time"].map(to_min)

# === 4) Filtrer la fenêtre 07:00–10:00 (bornes inclusives/exclusives au choix) ===
mask_window = (st["t_min"] >= 7*60) & (st["t_min"] < 10*60)
st_peak = st.loc[mask_window, ["trip_id","stop_id"]].dropna()

# === 5) Compter les trains par station ===
# N.B. on compte des trips DISTINCTS par stop_id pour éviter les doublons.
trains_par_station = (st_peak.drop_duplicates(["trip_id","stop_id"])
                            .groupby("stop_id", as_index=False)
                            .size()
                            .rename(columns={"size":"trains_7_10"}))

# === 6) Joindre les libellés (stop_name) et parent_station ===
stops_small = stops[["stop_id","stop_name","parent_station"]].copy()
res = trains_par_station.merge(stops_small, on="stop_id", how="left")

# Option : agréger au niveau "station" (parent_station) si les quais sont séparés
if "parent_station" in stops.columns and stops["parent_station"].notna().any():
    res_station = (res.assign(parent_station=res["parent_station"].fillna(res["stop_id"]))
                     .groupby("parent_station", as_index=False)["trains_7_10"].sum()
                     .merge(stops[["stop_id","stop_name"]], left_on="parent_station", right_on="stop_id", how="left")
                     .rename(columns={"stop_name":"station_name"})
                     .drop(columns=["stop_id"]))
    # res_station : trains par "station" (groupé)

res = res.sort_values(by="trains_7_10",ascending=False)
res

/tmp/ipykernel_2741/3766570262.py:32: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id


,stop_id,trains_7_10,stop_name,parent_station


Profil horaire RF

In [ ]:
df_ph_rf = pd.read_csv("data/validations/profil_horaire_rf.csv",sep=";")
profil_h_rf = df_ph_rf[["LIBELLE_ARRET","ID_ZDC","TRNC_HORR_60","Pourcentage_validations","CAT_JOUR"]]
profil_h_rf = profil_h_rf[profil_h_rf["CAT_JOUR"] == "JOHV"] #on s'intéresse aux jours ouvrés hors vacances (JOHV)

profil_7_10_rf = (profil_h_rf.loc[profil_h_rf["TRNC_HORR_60"].isin(["7H-8H","8H-9H","9H-10H"])]
               .groupby("ID_ZDC", as_index=False)["Pourcentage_validations"].sum()
               .rename(columns={"Pourcentage_validations": "part_7_10"}))
profil_7_10_rf["part_7_10"] = profil_7_10_rf["part_7_10"]*0.01

In [3]:
#Merge validations / profil_horaire
vald_prof_rf_1404 = vald_rf_1404.merge(profil_7_10_rf,on = "ID_ZDC",how="left")
vald_prof_rf_1404["NB_VALD_HP"] = (vald_prof_rf_1404["NB_VALD"]*vald_prof_rf_1404["part_7_10"]).round(0)

vald_prof_rf_1404 = vald_prof_rf_1404[["ID_ZDC","LIBELLE_ARRET","NB_VALD","part_7_10","NB_VALD_HP"]]
vald_prof_rf_1404.head(10)

NameError: name 'vald_rf_1404' is not defined

Lien GTFS / Validations en station

In [43]:
vald_rf_1404.head(10)

,JOUR,ID_ZDC,LIBELLE_ARRET,NB_VALD
10438,2025-04-14,71370.0,SAINT-LAZARE,257281.0
10396,2025-04-14,71264.0,CHATELET,151731.0
10633,2025-04-14,73626.0,GARE DE LYON,149435.0
10469,2025-04-14,71517.0,LA DEFENSE,139461.0
10357,2025-04-14,71139.0,MONTPARNASSE,123339.0
10447,2025-04-14,71410.0,GARE DU NORD,108211.0
10434,2025-04-14,71359.0,GARE DE L'EST,98297.0
10429,2025-04-14,71347.0,CH.D.G.ETOILE,64754.0
10670,2025-04-14,73794.0,LES HALLES,49564.0
10468,2025-04-14,71517.0,DEFENSE,49441.0


In [17]:
#Lien offre / validations pour stops : stop_id = IDFM:ID_ZDC pour metro/tram/bus, et IDFM:monomodalStopPlace pour RER/Transilien
ref_arrets = pd.read_csv("data/ref/ref_arrets.csv",sep=";")
ref_arrets["stop_id"] = "IDFM:" + str(ref_arrets["ArRId"])
ref_arrets = ref_arrets[["stop_id","ArRId"]]

ref_zone_arret = pd.read_csv("data/ref/ref_zone_arret.csv",sep=";")
ref_zone_arret["stop_id"] = "IDFM:monomodalStopPlace" + str(ref_zone_arret["ZdAId"])
ref_zone_arret = ref_zone_arret[["stop_id","ZdAId"]]

#Merge sur vald_rf_1404

vald_rf_1404["stop_id"] = ""

"""
freq_triée_metro = freq_triée_metro.merge(ref_arrets,on="stop_id",how="left")
freq_triée_train = freq_triée_train.merge(ref_zone_arret,on="stop_id",how="left")
"""


/tmp/ipykernel_2741/2721971081.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vald_rf_1404["stop_id"] = ""


'\nfreq_triée_metro = freq_triée_metro.merge(ref_arrets,on="stop_id",how="left")\nfreq_triée_train = freq_triée_train.merge(ref_zone_arret,on="stop_id",how="left")\n'

Partie Open street map : 
On va relier les stations IDFM à leur environnement urbain (POI OSM).
L'idée est d'utiliser stops.txt, sélectionner un sous-échantillon (au début, pour tester sans se faire bloquer par OSM), définir les catégories de POI à récupérer (boulangeries, écoles, restaurants, etc.), faire une boucle sur les stations, appeler ox.features_from_point(), compter les POI par catégorie dans deux rayons (400 m et 800 m), stocker les résultats dans une listeet créer un DataFrame final et sauvegarder en CSV

In [ ]:
!pip install geopandas osmnx requests shapely folium

In [ ]:
#Open street map 
#récupérer les points d’intérêt (écoles, commerces, hôpitaux, musées, stades, etc.) dans un rayon autour des stations.
import osmnx as ox 
#permet d’accéder directement aux données d’OpenStreetMap (OSM) : rues, bâtiments, POI (points d’intérêt), etc.
#Elle interroge automatiquement l’API Overpass, un service qui renvoie des données géographiques OSM au format GeoJSON.

tags = {'amenity': True, 'shop': True, 'tourism': True, 'leisure': True}
#Exemple des POI trouvés dans un rayon de 500m autour de Notre Dame de Paris
poi = ox.features_from_point((48.8566, 2.3522), dist=500, tags=tags)
poi.head()


In [ ]:
import pandas as pd
from tqdm import tqdm  # barre de progression

# --- 1. Lire le fichier stops.txt---
stops = pd.read_csv("data/gtfs/stops.txt", sep=",")  # sépateur CSV standard
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].dropna()

# Pour tester : prendre les 10 premières stations 
stops_sample = stops.head(10)

# --- 2. Définir les catégories de points d'intérêt ---
tags = {
    'amenity': True,   # cafés, écoles, hôpitaux...
    'shop': True,      # supermarchés, boulangeries...
    'leisure': True,   # parcs, piscines, stades...
    'tourism': True    # musées, monuments...
}

# --- 3. Fonction pour compter les POI autour d'une station ---
def count_poi_around_station(lat, lon, radius):
    try:
        gdf = ox.features_from_point((lat, lon), dist=radius, tags=tags)
        counts = {
            'n_amenities': gdf['amenity'].notna().sum(),
            'n_shops': gdf['shop'].notna().sum(),
            'n_leisure': gdf['leisure'].notna().sum(),
            'n_tourism': gdf['tourism'].notna().sum()
        }
        return counts
    except Exception as e:
        print(f"Erreur pour la station ({lat}, {lon}) : {e}")
        return {'n_amenities': 0, 'n_shops': 0, 'n_leisure': 0, 'n_tourism': 0}

# --- 4. Boucle principale ---
results = []

for _, row in tqdm(stops_sample.iterrows(), total=len(stops_sample)):
    lat, lon = row['stop_lat'], row['stop_lon']

    # POI dans un rayon de 400m
    c400 = count_poi_around_station(lat, lon, 400)
    # POI dans un rayon de 800m
    c800 = count_poi_around_station(lat, lon, 800)

    results.append({
        'stop_id': row['stop_id'],
        'stop_name': row['stop_name'],
        **{f'{k}_400m': v for k, v in c400.items()},
        **{f'{k}_800m': v for k, v in c800.items()}
    })

# --- 5. Résultats propres ---
poi_counts = pd.DataFrame(results)
print(poi_counts.head())

# --- 6. Sauvegarde ---
import os

# Crée le dossier si nécessaire
os.makedirs("Processed", exist_ok=True)
# Ensuite, on peut sauvegarder le CSV
poi_counts.to_csv("Processed/poi_counts_by_station.csv", sep=";", index=False)
print("Fichier enregistré : Processed/poi_counts_by_station.csv")

In [ ]:
# Lire le CSV généré
poi_counts = pd.read_csv("Processed/poi_counts_by_station.csv", sep=";")
# Afficher les 10 premières lignes
poi_counts.head(10)   

Attention : Ne pas tester directement sur toutes les stations tout de suite :
→ OSM risque de bloquer les requêtes après quelques dizaines d’appels.
→ On pourra ensuite écrire un script qui met une pause automatique entre les requêtes ou qui utilise un cache local.

Attention, le code suivant tourne pendant des heures, ne pas le lancer !!

In [ ]:
# ==============================
# Script complet POI + carte
# ==============================

import pandas as pd
import os
import time
from tqdm import tqdm
import folium
import osmnx as ox

# --- 1. Créer le dossier Processed si nécessaire ---
os.makedirs("Processed", exist_ok=True)

# --- 2. Lire stops.txt ---
stops = pd.read_csv("data/gtfs/stops.txt", sep=",")
stops.columns = stops.columns.str.strip()  # supprime espaces invisibles
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].dropna()
print(f"Nombre de stations : {len(stops)}")

# --- 3. Définir les catégories de POI ---
tags = {
    'amenity': True,
    'shop': True,
    'leisure': True,
    'tourism': True
}

# --- 4. Fonction pour compter les POI autour d'une station ---
def count_poi_around_station(lat, lon, radius):
    try:
        gdf = ox.features_from_point((lat, lon), dist=radius, tags=tags)
        counts = {
            'n_amenities': gdf['amenity'].notna().sum(),
            'n_shops': gdf['shop'].notna().sum(),
            'n_leisure': gdf['leisure'].notna().sum(),
            'n_tourism': gdf['tourism'].notna().sum()
        }
        return counts
    except Exception as e:
        print(f"Erreur pour la station ({lat}, {lon}) : {e}")
        return {'n_amenities': 0, 'n_shops': 0, 'n_leisure': 0, 'n_tourism': 0}

# --- 5. Boucle principale pour toutes les stations ---
results = []

for i, row in tqdm(stops.iterrows(), total=len(stops)):
    lat, lon = row['stop_lat'], row['stop_lon']

    # POI dans un rayon de 400m et 800m
    c400 = count_poi_around_station(lat, lon, 400)
    c800 = count_poi_around_station(lat, lon, 800)

    results.append({
        'stop_id': row['stop_id'],
        'stop_name': row['stop_name'],
        'stop_lat': lat,
        'stop_lon': lon,
        **{f'{k}_400m': v for k, v in c400.items()},
        **{f'{k}_800m': v for k, v in c800.items()}
    })

    # Pause 1 seconde pour ne pas surcharger l'API Overpass
    time.sleep(1)

    # Sauvegarde intermédiaire toutes les 50 stations
    if i % 50 == 0 and i > 0:
        temp_df = pd.DataFrame(results)
        temp_df.to_csv("Processed/poi_counts_by_station_temp.csv", sep=";", index=False)

# --- 6. Sauvegarde finale ---
poi_counts = pd.DataFrame(results)
poi_counts.to_csv("Processed/poi_counts_by_station.csv", sep=";", index=False)
print("Fichier final sauvegardé : Processed/poi_counts_by_station.csv")

# --- 7. Carte Folium interactive ---
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

for _, row in poi_counts.iterrows():
    folium.Marker(
        location=[row['stop_lat'], row['stop_lon']],
        popup=f"{row['stop_name']}\nAmenities 400m: {row['n_amenities_400m']}, Shops 400m: {row['n_shops_400m']}",
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(m)
    folium.Circle(location=[row['stop_lat'], row['stop_lon']], radius=400, color='green', fill=False).add_to(m)
    folium.Circle(location=[row['stop_lat'], row['stop_lon']], radius=800, color='red', fill=False).add_to(m)

# Sauvegarde la carte pour l’ouvrir dans le navigateur
m.save("Processed/poi_map.html")
print("Carte interactive sauvegardée : Processed/poi_map.html")


LECTURE DES POI

In [ ]:
#Cellule pour lire stop_times.txt stocké sur Google Drive car trop volumineux pour être stocké sur le dépôt GitHub
import os, gdown

#lien : https://drive.google.com/file/d/1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS/view?usp=sharing
file_id = "1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS"
url = f"https://drive.google.com/uc?id={file_id}"
os.makedirs("data", exist_ok=True)
out_path = "data/gtfs/stop_times.txt"
gdown.download(url, out_path, quiet=False)  # gère aussi les gros fichiers >100 Mo

stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id

Construction de la liste des stations

In [20]:
import pandas as pd
import geopandas as gpd

# -------------------------
# Lecture GTFS
# -------------------------
stops = pd.read_csv("data/gtfs/stops.txt")
routes = pd.read_csv("data/gtfs/routes.txt")
trips = pd.read_csv("data/gtfs/trips.txt")
stop_times = pd.read_csv("data/gtfs/stop_times.txt")

# -------------------------
# 1. Associer stop_id → route_type
# -------------------------
routes_small = routes[["route_id", "route_type"]]
trips_small = trips[["trip_id", "route_id"]]
stop_times_small = stop_times[["trip_id", "stop_id"]]

stop_trip = stop_times_small.merge(trips_small, on="trip_id", how="left")
stop_route = stop_trip.merge(routes_small, on="route_id", how="left")

stop_route = stop_route[["stop_id", "route_type"]].drop_duplicates()

# -------------------------
# 2. Filtrer métro / RER / tram / Transilien : types 0, 1, 2
# -------------------------
rail_like_types = [0, 1, 2]
stop_rail = stop_route[stop_route["route_type"].isin(rail_like_types)]

stops_rail = stops.merge(
    stop_rail[["stop_id"]].drop_duplicates(),
    on="stop_id",
    how="inner"
)

# -------------------------
# 3. Regrouper les quais : utiliser parent_station
# -------------------------
stops_rail["station_id"] = stops_rail["parent_station"].fillna(stops_rail["stop_id"])

stations = (
    stops_rail
    .groupby("station_id", as_index=False)
    .agg({
        "stop_name": "first",
        "stop_lon": "mean",
        "stop_lat": "mean"
    })
)

# -------------------------
# 4. Passage en GeoDataFrame pour ton analyse
# -------------------------
CRS_WGS84 = "EPSG:4326"
CRS_LAMBERT93 = "EPSG:2154"

stations_gdf = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["stop_lon"], stations["stop_lat"]),
    crs=CRS_WGS84
)



/tmp/ipykernel_2741/41847844.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("data/gtfs/trips.txt")
/tmp/ipykernel_2741/41847844.py:10: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv("data/gtfs/stop_times.txt")


In [39]:
stations_proj = stations_gdf.to_crs(CRS_LAMBERT93)
stations_proj = stations_proj.rename(columns={"stop_name": "station_name"})

print("Nb de stations métro/RER/tram/Transilien :", len(stations_proj))
stations_proj.head(10)
#liste de toutes les gares de métro / RER / transilien / tramway d'Ile de France avec leurs coordonnées en format LAMBERT93

Nb de stations métro/RER/tram/Transilien : 980


,station_id,station_name,stop_lon,stop_lat,geometry
0,IDFM:411281,Lycée Henri Sellier,2.515102,48.915963,POINT (664462 6868550.5)
1,IDFM:411284,Rougemont Chanteloup,2.514880,48.930999,POINT (664456 6870222.5)
2,IDFM:411318,Beauvais,2.088384,49.426191,POINT (633844.2 6925575.2)
3,IDFM:411327,Précy-sur-Oise,2.377341,49.203477,POINT (654617.865 6900596.5)
4,IDFM:411330,Saint-Leu-d'Esserent,2.417071,49.213795,POINT (657522 6901722)
5,IDFM:411333,Boran-sur-Oise,2.362238,49.166649,POINT (653484 6896509)
6,IDFM:411339,Trie-Château,1.818497,49.282401,POINT (614020.5 6909840)
7,IDFM:411343,Liancourt-Saint-Pierre,1.905479,49.220543,POINT (620254.5 6902868.5)
8,IDFM:411346,Chaumont-en-Vexin,1.873236,49.261190,POINT (617970 6907422.5)
9,IDFM:411349,Lavilletertre,1.920579,49.202382,POINT (621327 6900833.5)


In [41]:
vald_rf_1404.head(10)

,JOUR,ID_ZDC,LIBELLE_ARRET,NB_VALD,NB_VALD_MOY_JOUR,stop_id
10438,2025-04-14,71370,SAINT-LAZARE,257281.0,2827.0,IDFM:71370
10396,2025-04-14,71264,CHATELET,151731.0,1667.0,IDFM:71264
10633,2025-04-14,73626,GARE DE LYON,149435.0,1642.0,IDFM:73626
10469,2025-04-14,71517,LA DEFENSE,139461.0,1533.0,IDFM:71517
10357,2025-04-14,71139,MONTPARNASSE,123339.0,1355.0,IDFM:71139
10447,2025-04-14,71410,GARE DU NORD,108211.0,1189.0,IDFM:71410
10434,2025-04-14,71359,GARE DE L'EST,98297.0,1080.0,IDFM:71359
10429,2025-04-14,71347,CH.D.G.ETOILE,64754.0,712.0,IDFM:71347
10670,2025-04-14,73794,LES HALLES,49564.0,545.0,IDFM:73794
10468,2025-04-14,71517,DEFENSE,49441.0,543.0,IDFM:71517


In [ ]:
vald_rf_1404[vald_rf_1404["ID_ZDC"] == 74348.0] #ID de la gare de Gisors

#Donc certaines gares du GTFS ne sont pas présentes dans les données de validations

,JOUR,ID_ZDC,LIBELLE_ARRET,NB_VALD


In [46]:
#station_id = IDFM:ID_ZDC (ID_ZDC dans validations)
#On merge stations_proj avec les validations. On doit alors exclure les trams.

# -------------------------
rail_like_types = [1, 2]
stop_rail = stop_route[stop_route["route_type"].isin(rail_like_types)]

stops_rail = stops.merge(
    stop_rail[["stop_id"]].drop_duplicates(),
    on="stop_id",
    how="inner"
)

# -------------------------
# 3. Regrouper les quais : utiliser parent_station
# -------------------------
stops_rail["station_id"] = stops_rail["parent_station"].fillna(stops_rail["stop_id"])

stations = (
    stops_rail
    .groupby("station_id", as_index=False)
    .agg({
        "stop_name": "first",
        "stop_lon": "mean",
        "stop_lat": "mean"
    })
)

# -------------------------
# 4. Passage en GeoDataFrame pour ton analyse
# -------------------------
CRS_WGS84 = "EPSG:4326"
CRS_LAMBERT93 = "EPSG:2154"

stations_gdf = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["stop_lon"], stations["stop_lat"]),
    crs=CRS_WGS84
)

In [ ]:
stations_proj_rf = stations_gdf.to_crs(CRS_LAMBERT93)
stations_proj_rf = stations_proj_rf.rename(columns={"stop_name": "station_name"})

vald_rf_1404["ID_ZDC"] = vald_rf_1404["ID_ZDC"].astype(int)
vald_rf_1404["stop_id"] = "IDFM:" + vald_rf_1404["ID_ZDC"].astype(str)

stations_vald_rf = stations_proj_rf.merge(vald_rf_1404,left_on = "station_id",right_on="stop_id",how="inner")
#le inner permet d'exclure les gares non-présentes dans les validations
stations_vald_rf = stations_vald_rf[["station_id","station_name","stop_lon","stop_lat","geometry","NB_VALD","JOUR"]]
stations_vald_rf

/tmp/ipykernel_2741/256588747.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vald_rf_1404["ID_ZDC"] = vald_rf_1404["ID_ZDC"].astype(int)
/tmp/ipykernel_2741/256588747.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vald_rf_1404["stop_id"] = "IDFM:" + vald_rf_1404["ID_ZDC"].astype(str)


,station_id,station_name,stop_lon,stop_lat,geometry,NB_VALD,JOUR
0,IDFM:412687,Pernety,2.318359,48.834121,POINT (649964.47 6859556.75),7619.0,2025-04-14
1,IDFM:412697,Noisy-le-Grand - Mont d'Est,2.550027,48.840885,POINT (666974 6860187),25244.0,2025-04-14
2,IDFM:412992,Boucicaut,2.287856,48.841073,POINT (647732.5 6860349.5),8531.0,2025-04-14
3,IDFM:415852,Hôtel de Ville,2.351777,48.857470,POINT (652438.75 6862132.25),31384.0,2025-04-14
4,IDFM:422067,Le Val d'Or,2.216551,48.856254,POINT (642516 6862086.961),2691.0,2025-04-14
...,...,...,...,...,...,...,...
727,IDFM:73844,Champ de Mars Tour Eiffel,2.289656,48.856477,POINT (647880 6862061),10970.0,2025-04-14
728,IDFM:74001,Faidherbe - Chaligny,2.384051,48.850156,POINT (654800.35 6861300),7813.0,2025-04-14
729,IDFM:74002,Pont du Garigliano - Hôpital Européen G. Pompidou,2.270470,48.838883,POINT (646454.229 6860117.658),4877.0,2025-04-14
730,IDFM:74041,Front Populaire,2.365785,48.906714,POINT (653510.5 6867599.5),7466.0,2025-04-14


LOGEMENTS

In [ ]:
# --- 1) Charger les logements et décoder le WKB ---
log = pd.read_parquet("data/poi/poi_logement.parquet")  
gdf_log = gpd.GeoDataFrame(
    log,
    geometry=gpd.GeoSeries.from_wkb(log["geometry"]),  # <— WKB -> Shapely
    crs="EPSG:2154"  # <<< IMPORTANT : mets ici le CRS d'origine de tes logements !
)


# Si ce sont des polygones (bâtiments/parcelles) et que tu veux un comptage "au point",
# prends les centroïdes:
if gdf_logements.geom_type.isin(["Polygon", "MultiPolygon"]).any():
    gdf_logements = gdf_log.copy()
    gdf_logements["geometry"] = gdf_logements.geometry.centroid


gdf_logements = gdf_log.set_crs("EPSG:4326", allow_override=True) # 1) Dire à GeoPandas : "ce sont des degrés WGS84"

gdf_logements = gdf_logements.to_crs("EPSG:2154") # 2) Maintenant seulement : reprojection en Lambert-93

# --- 3) Construire les buffers autour des stations ---
rayon = 500 #rayon 500 mètres, on peut le changer

buffers = stations_proj[["station_id", "station_name", "geometry"]].copy()
buffers["geometry"] = buffers.buffer(rayon) 

# --- 4) Jointure spatiale : logements "dans" le buffer d'une station ---
# (predicate="within": chaque logement associé aux stations dont il est à l'intérieur du buffer)
hits = gpd.sjoin(gdf_logements[["geometry"]], buffers, how="inner", predicate="within")

# --- 5) Comptage par station ---
logt_par_station = (hits
    .groupby(["station_id", "station_name"], as_index=False)
    .size()
    .rename(columns={"size": "nb_logements"})
)

logt_par_station = logt_par_station.sort_values(by="nb_logements",ascending=False)
logt_par_station.head(5)

MONUMENTS

In [ ]:
monuments = gpd.read_file("data/poi/poi_monument.geojson")

monuments = monuments.set_crs("EPSG:4326", allow_override=True) # 1) Dire à GeoPandas : "ce sont des degrés WGS84"

gdf_monuments = monuments.to_crs("EPSG:2154") # 2) Maintenant seulement : reprojection en Lambert-93

# --- 3) Construire les buffers autour des stations ---
rayon = 500 #rayon 500 mètres, on peut le changer

buffers = stations_proj[["station_id", "station_name", "geometry"]].copy()
buffers["geometry"] = buffers.buffer(rayon) 

# --- 4) Jointure spatiale : monuments "dans" le buffer d'une station ---
# (predicate="within": chaque logement associé aux stations dont il est à l'intérieur du buffer)
hits = gpd.sjoin(gdf_monuments[["geometry"]], buffers, how="inner", predicate="within")

# --- 5) Comptage par station ---
monument_par_station = (hits
    .groupby(["station_id", "station_name"], as_index=False)
    .size()
    .rename(columns={"size": "nb_monuments"})
)

monument_par_station = monument_par_station.sort_values(by="nb_monuments",ascending=False)
monument_par_station.head(10)

Test sur les geojson/parquet

In [1]:
#Fonction simple pour charger un POI (GeoJSON ou Parquet)

import geopandas as gpd

def load_poi(path):
    if path.endswith(".parquet"):
        return gpd.read_parquet(path)
    else:
        return gpd.read_file(path)



In [2]:
#Aperçu d'un fichier de poi 

gdf = load_poi("data/poi/poi_monument.geojson")
gdf.head()


Skipping field duration: unsupported OGR type: 10
/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1149002 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


,element,id,heritage,heritage:operator,historic,mhs:inscription_date,name,ref:mhs,source:heritage,tourism,...,language:brochures:it,language:brochures:ja,language:brochures:nl,language:brochures:pl,language:brochures:pt,language:brochures:ru,language:brochures:zh,language:guides:fr,name:yi,geometry
0,node,191031796,None,None,milestone,None,Point zéro des Routes de France,None,None,attraction,...,None,None,None,None,None,None,None,None,None,POINT (2.3488 48.8534)
1,node,239899604,None,None,None,None,Miroir d’Eau,None,None,attraction,...,None,None,None,None,None,None,None,None,None,POINT (2.54967 48.85208)
2,node,247410298,3,mhs,castle,1948-01-27,Château de la Madeleine,PA00087405,Ministère de la Culture - 2022-08,attraction,...,None,None,None,None,None,None,None,None,None,POINT (2.04209 48.71005)
3,node,258615423,None,None,None,None,Vieux Pont de Limay,None,None,attraction,...,None,None,None,None,None,None,None,None,None,POINT (1.72565 48.99201)
4,node,258856843,None,None,None,None,Disneyland Railroad Main Street Station,None,None,attraction,...,None,None,None,None,None,None,None,None,None,POINT (2.77912 48.8708)


In [ ]:
!pip install folium

In [ ]:
#Visualisation des POI

import folium

m = folium.Map(location=[48.85, 2.35], zoom_start=11)

for _, row in gdf.sample(min(300, len(gdf))).iterrows():
    folium.CircleMarker(
        [row.geometry.y, row.geometry.x],
        radius=2
    ).add_to(m)

m
